## Insatcart implicit filtering model

The idea was from [Jess](https://jessesw.com/Rec-System/ ) blog with little modification. This model is based on colaborative filtering user-item model. Due to large size of the data, I have run the model in different group of customers. 

Two packages are needed to be updated on google colaborator. *italicized text*

In [ ]:
!pip install -U scikit-learn
!pip install implicit --upgrade

## Importing required packeges

In [ ]:
import pandas as pd
import numpy as np
import os,sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

import sklearn.metrics
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import implicit
import warnings
warnings.filterwarnings("ignore")

## Define functions

In [ ]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered 

In [ ]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    print('The size of user vector:',np.shape(user_vecs))
    print('The size of item vector:',np.shape(item_vecs))
    print('The size of recommendation vector:',np.shape(recommend_vector))
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.product_name.loc[item_lookup.product_id == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'product_id': codes, 'product_name': descriptions}) # Create a dataframe 
    
      
    return final_frame[['product_id', 'product_name']] # Switch order of columns around

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    #input()
    return metrics.auc(fpr, tpr)   

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    store_f1 = []  # An empty list to store the F1 score for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and stor
        #store_f1.append(metrics.f1_score(pred, actual))
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return [float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))] #,float('%.3f'%np.mean(store_f1))
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

## Imput files

In [ ]:
#The input directory in the google dirve
input_data_path="../Fromgoogle/Input_files"
os.listdir(input_data_path+'/produced')

## Making sparse matrix

In [ ]:
#purchases_sparse=sparse.load_npz(os.path.join(input_data_path+'/produced/','data_InstaCart_sparse_over100.npz'))
#rating=pd.read_csv(os.path.join(input_data_path+'/produced/','data_InstaCart_rating.csv'))

In [ ]:
data_rating=pd.read_csv(os.path.join(input_data_path+'/produced','data_InstaCart_rating.csv')).astype(int)

In [ ]:
#data_rating.loc[-1]=[0,0,0]
#data_rating.index = data_rating.index + 1  # shifting index
#data_rating = data_rating.sort_index()  # sorting by index

In [ ]:
data_rating.head()

In [ ]:
print(data_rating.info(),data_rating.shape)
data_rating.describe()

In [ ]:
data_cluster=pd.read_csv(os.path.join(input_data_path+'/produced','dprt_user_GMM_class.csv')).drop('Unnamed: 0',axis=1)
#cluster=pd.read_csv(os.path.join(input_data_path+'/produced','user_class_products.csv'))

In [ ]:
data_cluster.head(2)

In [ ]:
item_lookup=pd.read_csv(os.path.join('../Input_files/','products.csv'))[['product_id','product_name']]
item_lookup.head()

In [ ]:
def chosen_user(df_cluster,df_rating,grp,rank=None):
    print('***********Choosing a customer***********')
    if not rank: rank=10
    a=df_cluster.loc[df_cluster.user_dprt_class==grp,'user_id'].values
    b=df_rating.loc[df_rating.user_id.isin(a)].reset_index().drop('index',axis=1)
    c=pd.DataFrame({'quantity':b.groupby('user_id').size()}).reset_index()
    d=b.join(c.set_index('user_id'),on=('user_id'))
    print('The number of unique quantities in %ith cluster is %i'%(grp,d.quantity.nunique()))
    if d.quantity.nunique()<rank: rank=d.quantity.nunique()
    print('rank:',rank)
    #print(np.sort(d.quantity.unique()))
    indx=np.where(d.quantity==np.sort(d.quantity.unique())[rank])[0]
    #print(indx)
    #print(d.loc[indx,:].reset_index().loc[0,:])
    user=d.loc[indx,:].reset_index().loc[0,'user_id']
    print('Customer number %i has %ith place in cluster %i  of customers. \nThese customers were sroted by the totla number of ordered unique items.'%(user,rank,grp))
    
    return user


In [ ]:
chosen_user(data_cluster,data_rating,1,rank=None)

In [ ]:
def clustered_collab_filtering(user_place,group,cluster,df,num_items):
  #group: is the cluster number of the data you want to work on
  #cluster: is the dataset consist of user_id and user_dprt_class
  #df: is the data set containg user_id, product_id and rating 
  #user_place: The palce of the cutomer you have selected in the group based on 
  #            sorting by the number of unique items that every customer has ordered
  #num_items: How manyitems  do you want to recommend to the customer
    
  #returns: the list of recommended items and the mean AUC of the model
  
  #refining the data into asking cluster
  df=df.loc[df.user_id.isin(cluster.user_id[np.where(cluster.user_dprt_class==group)[0]]),:]
  print('\n\033[1mThe cluastering data:\033[0m\n',cluster.head(),cluster.info(),
        '\n\033[1mThe rating data for cluster %i:\033[0m\n'%group,df.head(),df.info())
  
  #-----------------------------------------------------------------------------------------------------------

  print('\033[1m*****Making sparse matrix for group '+str(group)+'*****\033[0m')
  customers = list(np.sort(df.user_id.unique())) # Get our unique customers
  products = list(df.product_id.unique()) # Get our unique products that were purchased
  quantity = list(df.rating) # All of our purchases

  rows = df.user_id.astype('category', categories = customers).cat.codes 
  # Get the associated row indices
  cols = df.product_id.astype('category', categories = products).cat.codes 
  # Get the associated column indices
  purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
      
  print(purchases_sparse.shape)
  
  matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
  num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
  sparsity = 100*(1 - (num_purchases/matrix_size))
  print('The sparsity of the data in cluster '+str(group)+' is: ',sparsity)
   
  #-----------------------------------------------------------------------------------------------------------

  print('\033[1m\n*****Making traing and test sets to evaluate*****\033[0m')
  product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)
   
  #-----------------------------------------------------------------------------------------------------------

  print('\033[1m\n*****Making user and item vecrots*****\033[0m')
  alpha = 15
  user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double')
                                                            ,factors=20,regularization = 0.1,iterations = 50)
  print('The size of user vector:',np.shape(user_vecs))
  print('The size of item vector:',np.shape(item_vecs))
  
  #-----------------------------------------------------------------------------------------------------------

  print('\033[1m\n*****Evaluation, calculating AUC*****\033[0m')
  evaluation=calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
  print('The AUC (area under the curve) is :',evaluation[0])
  #print('\n\n>>>>>>>>>>>>>>>>>>',df.info(),cluster.info())
  #-----------------------------------------------------------------------------------------------------------    
    
  print('\033[1m\n*****Making suggestion for the example for customer in cluster'+str(group)+'*****\033[0m')
  customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
  products_arr = np.array(products) # Array of product IDs from the ratings matrix
  print(df.info(),cluster.info(),group,user_place)
  selcted_user=chosen_user(df_cluster=cluster,df_rating=df,grp=group,rank=user_place)
  print('Selected user number:',selcted_user)
  result=rec_items(selcted_user,
                   purchases_sparse, user_vecs, item_vecs, 
                   customers_arr, products_arr, item_lookup,num_items = num_items)
  
  print('\033[1mThe final suggestions for customer %i :\033[0m \n'%selcted_user,result)
  
  return selcted_user

In [ ]:
grp=0
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place=user_place,group=grp,cluster=data_cluster,
                                        df=data_rating,num_items=10)
print('\n\033[1mProducts oredered by customer %i:\033[0m \n'%chosen_user)
print(item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
  join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),
       on=('product_id')).reset_index())

In [ ]:
grp=1
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place=user_place,group=grp,cluster=data_cluster,
                                        df=data_rating,num_items=10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id')
     ,on=('product_id')).reset_index()

In [ ]:
grp=2
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=3
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=4
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=5
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=6
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=7
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=8
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=9
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=10
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=11
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=12
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=13
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=14
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=15
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=16
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=17
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
grp=17
user_place=10   # the rank of sorted customer by the number of number of unique ordered products to choose (1,2,3,4,...,max), form  people who purchased least number of ordered procudts to most number of ordered products
chosen_user=clustered_collab_filtering(user_place,grp,data_cluster,data_rating,10)
print('\n\033[1mProducts oredered by customer %i\033[0m'%chosen_user)
item_lookup.loc[item_lookup.product_id.isin(data_rating.loc[data_rating.user_id==chosen_user].product_id)]. \
join(data_rating.loc[data_rating.user_id==chosen_user,['product_id','rating']].set_index('product_id'),on=('product_id')).reset_index()

In [ ]:
np.mean([0.956,0.865,0.864,0.905,0.884,0.916,0.873,0.892,0.84,0.923,0.906,0.947,0.854,0.85,0.938,0.839,0.733,0.92,0.921])